## Abstracting OSDU Request Service by inheriting the ConfigOsduEnvironment class.

Previouosly we created a ConfigOsduEnvironment Class in order to retrieve the variables needed to set up the OSDU client based on the environment we want to target (3. osdu_environment_config.ipynb).

Now, in the following notebook we create an OsduHttpClient class intended to be used to sent GET, POST, and PUT requestS to OSDU. The OsduHttpClient class receives the osdu environment name (data partition ids: "npequinor-test", "npequinor-dev", and "equinor-data") and a booleand which defines whether we want to use a public or private client_id, then we instantiate an OsduClient using the OSDU SDK, and create our request methods based on this last which recieve the relative API endpoint uri we want to target.

Finally we make a GET and POST requests to OSDU using our OsduHttpClient class.

In [ ]:
"""
REMARK:

This class is also moved to osdu_notebooks/libs/osdu_service/osdu_http_client.py
"""

In [1]:
from libs.osdu_service.osdu_environment_config import ConfigOsduEnvironment

from msal import ConfidentialClientApplication
from osdu.identity import OsduMsalNonInteractiveCredential, OsduMsalInteractiveCredential, OsduTokenCredential
from osdu.client import OsduClient

from typing import Union
import os

# Adding .env file variables as environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
class OsduHttpClient(ConfigOsduEnvironment):
    
    """
    Gets the configuration values according to the environment to target
    and the kind of authentication/authorization to be used and createS
    an OsduClient client that can be used to make requests to OSDU.
    
    osdu_environment: "npequinor-test", "npequinor-dev", or "equinor-data",
    client_type: "public-client", "private-client", or "token-client".
    The osdu_cli_client (public-client) set as default.
    """
    
    def __init__(self, osdu_environment:str, client_type="public-client"):
        ConfigOsduEnvironment.__init__(self, osdu_environment, client_type)
        if client_type == "public-client":
            self.osdu_client: OsduClient = self._get_osdu_public_client
        elif client_type == "private-client":
            self.osdu_client: OsduClient = self._get_osdu_private_client
        elif client_type == "token-client":
            self.osdu_client: OsduClient = self._get_osdu_token_client
    
    @property
    def _get_osdu_private_client(self):
        resource_id = self.env_variables['resource_id']
        scopes = f'{resource_id}/.default openid profile offline_access'
        authority = f"https://login.microsoftonline.com/{self.env_variables['tenant_id']}"

        private_app = ConfidentialClientApplication(
            client_id = self.env_variables['client_id'],
            client_credential = self.env_variables['client_secret'],
            authority = authority
            )

        credentials = OsduMsalNonInteractiveCredential(
            client_id = self.env_variables['client_id'],
            client_secret = self.env_variables['client_secret'],
            authority = authority,
            scopes = scopes,
            client = private_app
        )

        return OsduClient(
            server_url = self.env_variables['url'],
            data_partition = self.env_variables['data_partition_id'],
            credentials = credentials,
            retries = 5
        )
    
    @property
    def _get_osdu_public_client(self):
        resource_id = self.env_variables['resource_id']
        scopes = f'{resource_id}/.default openid'
        authority = f"https://login.microsoftonline.com/{self.env_variables['tenant_id']}"
        
        credentials = OsduMsalInteractiveCredential(
            client_id = self.env_variables['client_id'],
            authority = authority,
            scopes=scopes,
            token_cache="token_cache.txt"
        )
        
        return OsduClient(
            server_url = self.env_variables['url'],
            data_partition = self.env_variables['data_partition_id'],
            credentials=credentials
        )
        
    @property
    def _get_osdu_token_client(self):
        tenant_id = self.env_variables['tenant_id']
        token_endpoint = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
        
        credentials = OsduTokenCredential(
            client_id=self.env_variables['client_id'],
            token_endpoint=token_endpoint,
            refresh_token=os.environ[self.env_variables['refresh_token_key']],
            client_secret=self.env_variables['client_secret']
        )
        
        return OsduClient(
            server_url = self.env_variables['url'],
            data_partition = self.env_variables['data_partition_id'],
            credentials=credentials
        )
        
        
    def app_get_returning_json(self, service_relative_uri: str) -> dict:
        
        service_uri = f"{self.env_variables['url']}/{service_relative_uri}"
        response = self.osdu_client.get_returning_json(service_uri)
        
        return response
    
    def app_post_returning_json(self, service_relative_uri: str, payload:dict) -> dict:
        
        service_uri = f"{self.env_variables['url']}/{service_relative_uri}"
        response = self.osdu_client.post_returning_json(
            service_uri,
            payload
        )
        
        return response
    
    def app_put_returning_json(self, service_relative_uri: str, data: Union[str, dict], ok_status_codes: list = None):
        
        service_uri = f"{self.env_variables['url']}/{service_relative_uri}"
        if ok_status_codes is None:
                ok_status_codes = [201]
        response = self.osdu_client.put_returning_json(
            service_uri,
            data,
            ok_status_codes
        )
        return response
    
    def app_query_with_cursor(self, service_relative_uri:str, payload):
            
        cursor = ""
        results = []
        service_uri = f"{self.env_variables['url']}/{service_relative_uri}"
        while True:
            payload["cursor"] = cursor
            response = self.osdu_client.post_returning_json(
                service_uri,
                payload
            )
            if response["results"]:
                results.extend(response["results"])
            cursor = response.get["cursor"]
            if cursor is None:
                break
            
        return results    

In [3]:
"""
Instantiating our newly created class.
Test getting osdu_environment variables. Possible osdu_environment values are: "npequinor-test", "npequinor-dev", and "equinor-data"
"""
# client = OsduHttpClient("npequinor-dev", client_type="public-client")
# client = OsduHttpClient("npequinor-test", client_type = "public-client")
client = OsduHttpClient("equinor-data", client_type = "public-client")
client.env_variables

{'osdu_environment': 'equinor-data',
 'url': 'https://equinor.energy.azure.com/api',
 'resource_id': '5a1178c2-5867-4a34-8fb8-216164e30b5f',
 'client_id': '7a414874-4b27-4378-b34f-bc9e5a5faa4f',
 'client_secret': None,
 'data_partition_id': 'equinor-data',
 'tenant_id': '3aa4a235-b6e2-48d5-9195-7fcf05b459b0',
 'refresh_token_key': None}

In [4]:
"""
GET REQUEST: Listing legal tags.
"""

relative_url = "legal/v1/legaltags?valid=true"

res = client.app_get_returning_json(
    relative_url
)

res['legalTags'][0]

{'name': 'equinor-data-npequinor-dev-Wellbore-Legal-Tag-Test3130799',
 'description': 'Legal Tag added for Wellbore',
 'properties': {'countryOfOrigin': ['US', 'CA'],
  'contractId': '123456',
  'expirationDate': '2025-12-25',
  'originator': 'Schlumberger',
  'dataType': 'Public Domain Data',
  'securityClassification': 'Private',
  'personalData': 'No Personal Data',
  'exportClassification': 'EAR99'}}

In [5]:
"""
POST REQUEST: OSDU aggregated records by schema kind.
"""

payload = payload = {"kind": "*:*:*:*", 'aggregateBy': 'kind', 'query': "*"}
relative_url = "search/v2/query"

res = client.app_post_returning_json(
    relative_url,
    payload
)

res['aggregations'][:5]

[{'key': 'osdu:wks:dataset--File.Generic:1.0.0', 'count': 20383059},
 {'key': 'osdu:wks:work-product-component--WellLog:1.2.0', 'count': 1869510},
 {'key': 'eqnr:smda-api-v2.0:wellbore-alias:1.0.0', 'count': 803295},
 {'key': 'eqnr:smda-api-v2.0:wellheaders:1.0.0', 'count': 152908},
 {'key': 'osdu:wks:master-data--Wellbore:1.1.0', 'count': 152704}]